In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns
from pandas_profiling import ProfileReport
from datetime import datetime, timedelta
from sklearn import metrics
import math
from sklearn.metrics import classification_report
from sklearn import linear_model
import pickle as pkl
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy
import gradio as gr


In [3]:
df_completo_eda=pd.read_parquet(r"C:\Users\valen\Desktop\Henry\PI 01\MLOpsReviews\DFCREADOS\df_completo.parquet")

In [4]:
df_completo_eda.shape

(11024289, 17)

In [6]:
df_completo_eda.columns

Index(['userId', 'score', 'timestamp', 'id', 'show_id', 'type', 'title',
       'director', 'cast', 'country', 'date_added', 'release_year', 'rating',
       'duration', 'listed_in', 'description', 'source'],
      dtype='object')

In [8]:
DF_M01= df_completo_eda[['userId', 'id','score']]
reader = Reader(rating_scale=(1, 5))
info = Dataset.load_from_df(DF_M01, reader)

In [9]:
trainset, testset = train_test_split(info, test_size=0.25)

In [10]:
model = SVD()

In [11]:
model.fit(trainset)

In [12]:
valores_predict = model.test(testset)

In [14]:
rmse = accuracy.rmse(valores_predict)
mae = accuracy.mae(valores_predict)

print(f"RMSE: {rmse}")
print(f"MAE: {mae}")

RMSE: 1.0013
MAE:  0.7764
RMSE: 1.001337397369716
MAE: 0.7763798508428689


In [16]:
def modelo_prediccion(userId, id):
    prediction = model.predict(userId, id)

    if prediction.est >= 4:
      mensaje = "Perfecta para vos!!"
    elif prediction.est >= 3 and prediction.est < 4:
      mensaje = "Te la recomendamos"
    else:
      mensaje = "No te la recomendamos"
    return mensaje


iface = gr.Interface(fn=modelo_prediccion, 
                     inputs= [gr.inputs.Textbox(lines=1,placeholder="Usuario"), 
                             gr.inputs.Textbox(lines=1,placeholder="Codigo de Pelicula")],
                     outputs="text")
iface.launch()

C:\Users\valen\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\gradio\inputs.py:27: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
C:\Users\valen\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\gradio\deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
C:\Users\valen\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\gradio\deprecation.py:40: UserWarning: `numeric` parameter is deprecated, and it has no effect
  warnings.warn(value)


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [19]:
with open('M01_Entrenado.pkl', 'wb') as archivo:
    pkl.dump(model, archivo)